In [303]:
# !pip install keras
# !pip install tensorflow
# !pip install xgboost

In [154]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [ ]:
xls = pd.ExcelFile('2 years data.xlsx',engine='openpyxl')
df1 = pd.read_excel(xls, 'Planilha1')
df2 = pd.read_excel(xls, 'Price increase dates')
df3 = pd.read_excel(xls, 'Mandatory vacation dates',names=['start','end'])

df1 = df1.iloc[:245815]
df2.dropna(axis=1,inplace=True)
df3.dropna(inplace=True)
df3.drop([0],inplace=True)
df1.drop(['Order nº','Custumer Name','Item Name','Subgroup_description'],axis=1,inplace=True)
df2.drop(['Subgroup of products (description)'],axis=1,inplace=True)

le = preprocessing.LabelEncoder()
for col in ['Country','State','City']:
    df1[col] = le.fit_transform(df1[[col]])
df = df1.groupby(['Order date','Custumer Id','Item Id','Country','State','City','Subgruop_ID']).mean().reset_index()

## recency, frequency, price_increase_30, next_buy
### is_vacation, vacation_30

In [142]:
def time_series_bin(right,col):

    ret = pd.DataFrame(pd.date_range('2019-10-01','2021-09-13',freq='d'),columns=['Date'])
    try:
        new_data = pd.merge(ret,right,left_on='Date',right_on=col,how='outer')
    except:
        new_data = pd.concat(ret,right,axis=1,join="outer",)
    new_data['ret']= new_data[col].isna()==0
    new_data['ret'].replace({True:1,False:0},inplace=True)
    return new_data['ret']

In [145]:
def recency(dfr,cusid):

    t = time_series_bin(dfr.loc[dfr['Custumer Id']==cusid][['Order date']].drop_duplicates(),'Order date')
    t = np.where(t.to_numpy()==1)[0]
    t1 = [t[i]-t[i+1] for i in range(len(t)-1)]
    tt = [0-t[0]]
    [tt.append(i) for i in t1]
    tt.append(t[-1]-713)

    data = []
    for rec in tt :
        [data.append(_) for _ in range(abs(rec))]

    return data

In [152]:
def buy_freq(dfr,cusid):
    t = time_series_bin(dfr.loc[dfr['Custumer Id']==cusid][['Order date']].drop_duplicates(),'Order date')
    ret = [0]
    [ret.append(t[:i+1].mean()) for i in range(len(t)-1)]
    
    return ret

In [ ]:
def price_inc_30():
    df2 = pd.read_excel(xls, 'Price increase dates')
    df2.dropna(axis=1,inplace=True)
    df2.drop(['Subgroup of products (description)'],axis=1,inplace=True)
    df2 = pd.concat([df2[['Date of price increase']]
                     ,pd.DataFrame.sparse.from_spmatrix((OneHotEncoder().fit_transform(df2.drop(['Date of price increase']
                                                                                                ,axis=1))))
                    ]
                    ,axis=1)
    df2 = df2.groupby(['Date of price increase']).sum()

    x = pd.DataFrame(index=pd.date_range('2019-10-01','2021-09-13',freq='d'))
    x = pd.concat([x,df2],join="outer",axis=1,)
    x.fillna(0,inplace=True)
    return x

In [ ]:
def next_buy(dfr,cusid):
    t = time_series_bin(dfr.loc[dfr['Custumer Id']==cusid][['Order date']].drop_duplicates(),'Order date')
    t = np.where(t.to_numpy()==1)[0]
    t1 = [t[i+1]-t[i] for i in range(len(t)-1)]
    t2 = []
    tt = [t[0]]
    [tt.append(i) for i in t1]
    tt.append(713-t[-1])
    
    for i in range(len(tt)-1):
        for j in range(tt[i]):
            t2.append(tt[i]-j-1)
    
    for i in range(713-len(t2)):
        t2.append(90)
    return(t2)

In [ ]:
def data_cus(cusid):
    